# HuggingFace Pipeline

In [31]:
!pip install -q transformers datasets
!pip install -q sentencepiece
!pip install -q kobert-transformers
!pip install -q python-dotenv

In [32]:
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore') # 모든 경고 무시


load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')


### NLP Tasks

주어진 task에서 사전 훈련된 모델을 사용하는 가장 간단한 방법은 `pipeline`을 사용하는 것이다.

In [33]:
from transformers import pipeline

##### 기계번역

https://huggingface.co/Helsinki-NLP/opus-mt-ko-en

pipeline() : 항상 Hugging Face Hub에 있는 모델을 기본으로 사용
- 모델을 명시하지 않는 경우
    - HuggingFace가 task별 기본(default) 모델을 자동 사용
    - 이 기본모델들은 HuggingFace Hub에 공개된 모델
- 모델을 명시한 경우
    - HuggingFace에 올라온 특정 repo에서 다운로드
- 모델 객체를 직접 넘길 경우
    - 로컬에서 로드
    - 직접 학습/파인튜닝한 모델
    - 이런 경우에는 pipeline이 "추론 레포"의 역할만 한다.

In [34]:
# !pip install tf-keras

In [35]:
translator = pipeline('translation', model='Helsinki-NLP/opus-mt-ko-en')
translator

Device set to use cpu


In [36]:
translator('핫도그를 향하여!!!')

[{'translation_text': 'Turn to the hot dog!'}]

In [37]:
translator([
    '공부를 열심히 하시면 좋겠어요.',
    '수업을 열심히 들으시면 좋겠고요.',
    '실습도 열심히 해주시면 좋겠어요.',
    '제발 23기 말 좀... 들으세요...'

])

[{'translation_text': 'I hope you study hard.'},
 {'translation_text': "I'd like you to take the class really hard."},
 {'translation_text': "I'd like you to work hard on your practice."},
 {'translation_text': 'Please listen to the 23rd, please.'}]

##### 감성 분석

In [38]:
sentiment_clf = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [39]:
sentiment_clf("I'm very happy!!!")

[{'label': 'POSITIVE', 'score': 0.9998722076416016}]

- 한국어

https://huggingface.co/sangrimlee/bert-base-multilingual-cased-nsmc

In [40]:
ko_sentiment_clf = pipeline('sentiment-analysis', model='sangrimlee/bert-base-multilingual-cased-nsmc')

Device set to use cpu


In [41]:
ko_sentiment_clf([
    '어제 배운 Attention이 어려워서 맥주를 한 잔 했어.',
    '공부는 어렵지만 맥주는 시원하더라',
    '하지만 나는 굴하지 않고 열심히 공부할거야!!!'
])

[{'label': 'negative', 'score': 0.9047993421554565},
 {'label': 'negative', 'score': 0.7013086676597595},
 {'label': 'positive', 'score': 0.7580994367599487}]

##### Zero Shot Classification

- shot == 예시
- 예시 없이 (학습 없이) 추론(분류)

In [42]:
# 제로샷 분류 파이프라인(기본 NLI 모델 자동 로드)
zero_shot_clf = pipeline('zero-shot-classification')

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [43]:
zero_shot_clf(
    "This is a course about the transformers library.",
    candidate_labels=["education", "politics", "business"]
)

{'sequence': 'This is a course about the transformers library.',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.9053581953048706, 0.07259626686573029, 0.022045595571398735]}

In [44]:
zero_shot_clf(
    "It has a poison and it's very dangerous",
    candidate_labels=["tiger", "squirrel", "snake"]
)

{'sequence': "It has a poison and it's very dangerous",
 'labels': ['snake', 'tiger', 'squirrel'],
 'scores': [0.6373922228813171, 0.19870398938655853, 0.16390381753444672]}

- 한국어

https://huggingface.co/joeddav/xlm-roberta-large-xnli

In [45]:
ko_zero_shot_clf = pipeline('zero-shot-classification', model='joeddav/xlm-roberta-large-xnli')

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [46]:
sequence_to_classify = "2025년에는 어떤 운동을 하시겠습니까?!"
candidate_labels = ["정치", "경제", "건강"]

ko_zero_shot_clf(sequence_to_classify, candidate_labels)

{'sequence': '2025년에는 어떤 운동을 하시겠습니까?!',
 'labels': ['정치', '건강', '경제'],
 'scores': [0.40395545959472656, 0.3669034242630005, 0.22914111614227295]}

In [ ]:
sequence_to_classify = "2025년에는 어떤 운동을 하시겠습니까?!"
candidate_labels = ["정치", "경제", "건강"]
hypothesis_template = "이 텍스트는 {}에 관한 내용입니다."   # NLI 가설 문장 템플릿

ko_zero_shot_clf(sequence_to_classify,                  # 문장
                candidate_labels,                       # 후보    
                hypothesis_template=hypothesis_template # 사용자 정의 가설 문장
)

{'sequence': '2025년에는 어떤 운동을 하시겠습니까?!',
 'labels': ['건강', '정치', '경제'],
 'scores': [0.9117788672447205, 0.05924064293503761, 0.02898053266108036]}

##### 텍스트 생성

In [48]:
text_generator = pipeline('text-generation', model='distilgpt2')

Device set to use cpu


In [ ]:
text_generator(
    "In this course, we will teach you how to", # 텍스트 생성 시작 프롬프트
    max_length=30,                              # 생성 결과 최대 토큰 길이
    num_return_sequences=2                      # 서로 다른 생성 결과 2개 반환
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'In this course, we will teach you how to do this on the web. We will also teach you how to write a beautiful, beautiful, practical language. These are all new concepts in the course, so please check them out.'},
 {'generated_text': 'In this course, we will teach you how to get started on your own time and on your own time.'}]

- 한국어

https://huggingface.co/skt/ko-gpt-trinity-1.2B-v0.5

In [50]:
ko_text_generator = pipeline('text-generation', model='skt/ko-gpt-trinity-1.2B-v0.5')

Device set to use cpu


In [51]:
# ko_text_generator('AI란 말입니다')
ko_text_generator('오늘 점심에는')

[{'generated_text': '오늘 점심에는 뭐 먹었어\n 답변:그럼요. 식사 잘 하셨어요? 음식은 되도록 싱겁게 먹는게 건강에 좋대요.'}]

##### Fill Mask

In [52]:
unmasker = pipeline('fill-mask')

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [53]:
unmasker('This course will teach you all about <mask> model.', top_k=3)

[{'score': 0.09073042869567871,
  'token': 265,
  'token_str': ' business',
  'sequence': 'This course will teach you all about business model.'},
 {'score': 0.072625532746315,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical model.'},
 {'score': 0.049304164946079254,
  'token': 5,
  'token_str': ' the',
  'sequence': 'This course will teach you all about the model.'}]

##### NER

In [54]:
ner = pipeline('ner', model='dslim/bert-base-NER', grouped_entities=True)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [55]:
ner('My name is Squirrel and I work at Playdata in Seoul')

[{'entity_group': 'PER',
  'score': np.float32(0.98995215),
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity_group': 'PER',
  'score': np.float32(0.46767735),
  'word': '##qui',
  'start': 12,
  'end': 15},
 {'entity_group': 'PER',
  'score': np.float32(0.72662497),
  'word': '##rrel',
  'start': 15,
  'end': 19},
 {'entity_group': 'ORG',
  'score': np.float32(0.986338),
  'word': 'Playdata',
  'start': 34,
  'end': 42},
 {'entity_group': 'LOC',
  'score': np.float32(0.9994649),
  'word': 'Seoul',
  'start': 46,
  'end': 51}]

##### Q&A

In [56]:
qna = pipeline('question-answering')

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [57]:
qna(
    question="Where do I work?",
    context="My name is Squirrel and I work at Playdata in Seoul"
)

{'score': 0.8710189713747241, 'start': 34, 'end': 42, 'answer': 'Playdata'}

- 한국어

In [58]:
ko_qna = pipeline('question-answering', model='klue/roberta-base')

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [59]:
ko_qna(
    question="1931년 어떤 진단을 받았나요?",
    context="""그러던 1931년, 그는 갑작스럽게 폐결핵을 진단받았다. 이상은 하루에 담배를 50개피 이상 피는 것을 자신의 일과라고 표현했을 정도로 엄청난 골초였는데, 그 때문인지 병세는 날이 갈수록 악화되어 담당의가 그의 폐를 확인하고는 형체도 안 보인다며 혀를 내둘렀을 정도였다. 결국 1933년부터는 각혈까지 시작되었고, 건축기사 일을 지속하기 어렵다고 판단한 이상은 조선총독부에서 퇴사하고 황해도에 있는 배천 온천으로 요양을 간다."""
)

{'score': 0.0002860031963791698, 'start': 138, 'end': 140, 'answer': '혀를'}

In [60]:
help(ko_qna)

Help on QuestionAnsweringPipeline in module transformers.pipelines.question_answering object:

class QuestionAnsweringPipeline(transformers.pipelines.base.ChunkPipeline)
 |  QuestionAnsweringPipeline(model: Union[ForwardRef('PreTrainedModel'), ForwardRef('TFPreTrainedModel')], tokenizer: transformers.tokenization_utils.PreTrainedTokenizer, modelcard: Optional[transformers.modelcard.ModelCard] = None, framework: Optional[str] = None, task: str = '', **kwargs)
 |
 |  Question Answering pipeline using any `ModelForQuestionAnswering`. See the [question answering
 |  examples](../task_summary#question-answering) for more information.
 |
 |  Example:
 |
 |  ```python
 |  >>> from transformers import pipeline
 |
 |  >>> oracle = pipeline(model="deepset/roberta-base-squad2")
 |  >>> oracle(question="Where do I live?", context="My name is Wolfgang and I live in Berlin")
 |  {'score': 0.9191, 'start': 34, 'end': 40, 'answer': 'Berlin'}
 |  ```
 |
 |  Learn more about the basics of using a pipeli